# Bitcoin Mint Verification Notebook
This notebook allows you to fetch a Natural Standard image from a short-lived URL, compute its SIFT features, generate the Bitcoin mint address, and display the result. Simply run each cell in order.

In [ ]:
!pip install opencv-python==4.8.0.76 opencv-contrib-python==4.8.0.76 base58==2.1.1 numpy==1.26.4

In [ ]:
import cv2import numpy as npimport hashlibimport binasciiimport base58import requestsfrom IPython.display import display, HTML

In [ ]:
# --- Define helper functions for SIFT and address generation ---
def compute_sift_features_from_array(gray_image):
    # Normalize lighting
    img = cv2.equalizeHist(gray_image)
    sift = cv2.SIFT_create(contrastThreshold=0.04, edgeThreshold=10, nfeatures=200)
    keypoints, descriptors = sift.detectAndCompute(img, None)
    if descriptors is None:
        feature_vector = np.array([])
    else:
        feature_vector = descriptors.flatten()
    return feature_vector

def generate_feature_vector_hash(feature_vector):
    counter = 0
    while True:
        rolled = np.roll(feature_vector, counter)
        fv_bytes = rolled.tobytes()
        h = hashlib.sha256(fv_bytes).hexdigest()[:19]
        if '0' not in h and 'I' not in h:
            return h
        counter += 1

def b58ec(hex_str):
    data = bytearray.fromhex(hex_str)
    return base58.b58encode(data).decode('ascii')

def b58dc(encoded, trim=0):
    raw = base58.b58decode(encoded)
    return raw[:-trim] if trim else raw

def hh256(data_bytes):
    first = hashlib.sha256(data_bytes).digest()
    second = hashlib.sha256(first).digest()
    return binascii.hexlify(second)

def burn(s):
    decoded = b58dc(s, trim=4)
    hexed = binascii.hexlify(decoded).decode('ascii')
    check = hh256(decoded)[:8].decode('ascii')
    return b58ec(hexed + check)

## Fetch and Verify

In [ ]:
# Prompt for the short-lived URL of the uploaded Natural Standard image
url = input("Enter the short-lived image URL:")
# Download image into memory
resp = requests.get(url)
in_memory = np.frombuffer(resp.content, dtype=np.uint8)
gray = cv2.imdecode(in_memory, cv2.IMREAD_GRAYSCALE)
# Compute SIFT feature vector and hash
fv = compute_sift_features_from_array(gray)
fv_hash = generate_feature_vector_hash(fv)
template = f'1BtcMint{fv_hash}XXXXXXX'
address = burn(template)
print(f'Computed Bitcoin Mint Address: {address}')
# (Optional) Display keypoint visualization if desired
# kp_img = cv2.drawKeypoints(cv2.equalizeHist(gray), cv2.SIFT_create(contrastThreshold=0.04, edgeThreshold=10, nfeatures=200).detect(gray, None)[0], None)
# _, buf = cv2.imencode('.png', kp_img)
# display(HTML(f'<img src="data:image/png;base64,{base64.b64encode(buf).decode()}" style="max-width:300px;border:1px solid #ccc;"/>'))
